In [ ]:
import pdb

In [ ]:
import loom

In [ ]:
c, d = loom.load()

In [ ]:
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource

from bokeh.palettes import Spectral10

from bokeh.models import HoverTool, BoxZoomTool, PanTool, WheelZoomTool, Callback

output_notebook()

In [ ]:
spectral_network = d[0]
walls = []
labels = {'branch_points': [], 'joints': [], 'walls': []}
for i, s_wall in enumerate(spectral_network.s_walls):
    segments = []
    seg_labels = []
    split_at = []

    z_segs = [s_wall.z]

    for z_seg in z_segs:
        segments.append([z_seg.real, z_seg.imag])
        seg_labels.append(s_wall.label)

    walls.append(segments)
    labels['walls'].append(seg_labels)

In [ ]:
hover = HoverTool(
    always_active=False,
    line_policy="nearest",
    point_policy="follow_mouse",
    tooltips=[
        ("index", "$index"),
        ("(x,y)", "($x, $y)"),
        ("label", "@label"),
    ]
)

p = figure(tools=[BoxZoomTool(), PanTool(), WheelZoomTool(), hover])

for i, wall in enumerate(walls):
    for j, segment in enumerate(wall):
        seg_xs, seg_ys = segment
        source = ColumnDataSource(
            data=dict(
                x=seg_xs,
                y=seg_ys,
                label=[labels['walls'][i][j] for x in seg_xs],
            )
        )
        p.line('x', 'y', line_color=Spectral10[(i+j) % 10], source=source)
show(p)

In [ ]:
plots = []

for sn in d:    
    p = figure()
    #for i, s_wall in enumerate(sn.s_walls):
    for s_wall in sn.s_walls:
        p.line(s_wall.z.real, s_wall.z.imag, 
               #line_color=Spectral10[i % 10]
              )
        text_x = s_wall.z.real[-1]
        text_y = s_wall.z.imag[-1]
        text_str = s_wall.label
        p.text([text_x], [text_y], [text_str])
    plots.append(p)
    
#show(plots[0])

In [ ]:
def show_plot(plot_index):
    show(plots[plot_index])

In [ ]:
from IPython.html.widgets import IntSlider, interactive
from IPython.display import display
plot_slider = interactive(show_plot, plot_index=IntSlider(min=0, max=len(d)-1, step=1, value=0))
display(plot_slider)